Sources
* Data: https://www.ntia.doc.gov/page/download-digital-nation-datasets
* Map (useful for checks!): https://www.ntia.doc.gov/data/digital-nation-data-explorer#sel=internetUser&disp=map
* Docs: https://www.ntia.doc.gov/files/ntia/publications/november-2019-techdocs.pdf
* Universes: https://www.ntia.doc.gov/files/ntia/data_central_downloads/code/create-ntia-tables-stata.zip


Useful variables:
* peinhome
* peinwork
* pemphone
* pemphone
* hemobdat
* hehomte1
* hefaminc
* pesex
* prtage
* peeduca 
* ptdtrace
* prdthsp
* prnmchld
* gestfips


For the weights, the 2019 sample documentation suggests `PWSSWGT` and `HWHHWGT` by default, and `PWPRMWGT` for the personal questions.

In [1]:
cps_test = pd.read_csv("data/nov19-cps.csv")

# Californian people...
ca = cps_test.query("(gestfips == 6) & (prtage >= 3) & (prpertyp != 3)")

ca.query("peinhome == 1").pwprmwgt.sum() / ca.pwprmwgt.sum(), \
ca.query("peinhome == 1").pwsswgt.sum() / ca.pwsswgt.sum() 

(0.9328654188139469, 0.7235029866408134)

So `pwsswgt` is right in this case -- it is not strictly within the used services category (email, texts, social, video, etc.).  Pay attention to exactly what it takes to reproduce the results on the official site.

In [2]:
cps = pd.read_csv("data/nov19-cps.csv")

cps = cps.filter(regex = "^((?!wgt[0-9]).)*$", axis = 1)

## Construct all the universe, following their cuts.
cps["isPerson"]      = (cps.prtage >= 3) & (cps.prpertyp != 3)
cps["isHouseholder"] = (cps.perrp > 0) & (cps.perrp < 3) & (cps.hrhtype > 0) & (cps.hrhtype < 9)
cps["isAdult"]       = cps.isPerson & (cps.prtage >= 15)
cps["isRespondent"]  = cps.puelgflg == 20

cps["P_at_home"]     = np.where(cps.isPerson, cps.peinhome == 1, np.nan)
cps["P_at_work"]     = np.where(cps.isPerson, cps.peinwork == 1, np.nan)
cps["P_at_school"]   = np.where(cps.isPerson, cps.peinschl == 1, np.nan)
cps["P_anywhere"]    = np.where(cps.isPerson, 
                                (cps.peinhome == 1) | (cps.peinwork == 1) | (cps.peinschl == 1) | \
                                (cps.peincafe == 1) | (cps.peintrav == 1) | (cps.peinlico == 1) | \
                                (cps.peinelho == 1) | (cps.peinothr == 1), 
                                np.nan)

cps["P_mobdat"]      = np.where(cps.isPerson, cps.hemobdat == 1, np.nan)

cps["P_computer"]    = np.where(cps.isPerson, (cps.pelaptop == 1) | (cps.pedesktp == 1), np.nan)
cps["P_smartphone"]  = np.where(cps.isPerson, cps.pemphone == 1, np.nan)

cps["P_email"]       = np.where(cps.isPerson, cps.peemail == 1, np.nan)
cps["P_text"]        = np.where(cps.isPerson, cps.petextim == 1, np.nan)
cps["P_social"]      = np.where(cps.isPerson, cps.pesocial == 1, np.nan)
cps["P_video"]       = np.where(cps.isPerson, cps.pevideo == 1, np.nan)
cps["P_services"]    = np.where(cps.isPerson, cps.peusesvc == 1, np.nan)


cps["H_at_home"]     = np.where(cps.isHouseholder, cps.heinhome == 1, np.nan)
cps["H_anywhere"]    = np.where(cps.isHouseholder, 
                                (cps.heinhome == 1) | (cps.heinwork == 1) | (cps.heinschl == 1) | \
                                (cps.heincafe == 1) | (cps.heintrav == 1) | (cps.heinlico == 1) | \
                                (cps.heinelho == 1) | (cps.heinothr == 1), 
                                np.nan)

# Reaons for non-use.
cps["H_nonuse_cost"] = np.where(cps.isHouseholder, cps.heprinoh.isin([2, 3]), np.nan)
cps["H_nonuse_no_interest"] = np.where(cps.isHouseholder, cps.heprinoh == 1, np.nan)
cps["H_nonuse_not_available"] = np.where(cps.isHouseholder, cps.heprinoh == 5, np.nan)

cps["H_smartphone"]  = cps.hemphone == 1
cps["H_computer"]    = (cps.helaptop == 1) | (cps.hedesktp == 1)

## This is NOT how they construct the variable -- 
##   they do high speed conditional on internet.
cps["H_highsp"] = np.where(cps.isHouseholder, cps.hehomte1 == 1, np.nan)
cps["H_mobdat"] = np.where(cps.isHouseholder, cps.hemobdat == 1, np.nan)

## So to reproduce their numbers, we'd do this (checked!!).
## cps.query("isHouseholder & (H_anywhere == True)").groupby("state").apply(weighted_mean, w = "hwhhwgt").H_mobdat
## cps.query("isHouseholder & (H_at_home == True)") .groupby("state").apply(weighted_mean, w = "hwhhwgt").H_highsp

### Lots of contortions to get Census places.

Start with states, so we can check against "official" sources.

In [3]:
fips = pd.read_sql("SELECT fips, name FROM states;", index_col = "fips", con = cen_con).name.to_dict()
cps["state"] = cps.gestfips.replace(fips)

cps_ca = cps.query("gestfips == 6")

Population counts look right.

In [4]:
cps.groupby("state").pwsswgt.sum().loc[["Illinois", "California", "Pennsylvania"]] / 1e6

state
Illinois        12.497291
California      39.251506
Pennsylvania    12.622756
Name: pwsswgt, dtype: float64

In [5]:
cities = ["new_york", "los_angeles", "chicago", "houston", "phoenix",
          "philadelphia", "san_antonio", "san_diego", "dallas",
          "san_jose", "austin", "jacksonville", "san_francisco", "columbus",
          "fort_worth", "indianapolis", "charlotte", "seattle", "denver", "washington"]

city_dict = { # state, cbsa, city #
  "new_york"     : {"gestfips" : 36, "gtcbsa" : 35620, "gtcbsast" : 1, "gtindvpc" : 1}, 
  "los_angeles"  : {"gestfips" :  6, "gtcbsa" : 31080, "gtcbsast" : 1, "gtindvpc" : 1, "gtco" :  37}, 
  "chicago"      : {"gestfips" : 17, "gtcbsa" : 16980, "gtcbsast" : 1, "gtindvpc" : 1}, 
  "houston"      : {"gestfips" : 48, "gtcbsa" : 26420, "gtcbsast" : 1, "gtindvpc" : 1}, 
  "phoenix"      : {"gestfips" :  4, "gtcbsa" : 38060, "gtcbsast" : 1, "gtindvpc" : 1},
  "philadelphia" : {"gestfips" : 42, "gtcbsa" : 37980, "gtcbsast" : 1, "gtindvpc" : 0, "gtco" : 101},
  "san_antonio"  : {"gestfips" : 48, "gtcbsa" : 41700, "gtcbsast" : 1, "gtindvpc" : 0}, 
  "san_diego"    : {"gestfips" :  6, "gtcbsa" : 41740, "gtcbsast" : 1, "gtindvpc" : 1}, 
  "dallas"       : {"gestfips" : 48, "gtcbsa" : 19100, "gtcbsast" : 1, "gtindvpc" : 1},
  "san_jose"     : {"gestfips" :  6, "gtcbsa" : 41940, "gtcbsast" : 1, "gtindvpc" : 1}, 
  "austin"       : {"gestfips" : 48, "gtcbsa" : 12420, "gtcbsast" : 1, "gtindvpc" : 0}, 
  "jacksonville" : {"gestfips" : 12, "gtcbsa" : 27260, "gtcbsast" : 1, "gtindvpc" : 0}, 
  "san_francisco": {"gestfips" :  6, "gtcbsa" : 41860, "gtcbsast" : 1, "gtindvpc" : 1, "gtco" :  75}, # SF county
  "columbus"     : {"gestfips" : 39, "gtcbsa" : 18140, "gtcbsast" : 1, "gtindvpc" : 0},
  "fort_worth"   : {"gestfips" : 48, "gtcbsa" : 19100, "gtcbsast" : 1, "gtindvpc" : 2}, 
  "indianapolis" : {"gestfips" : 18, "gtcbsa" : 26900, "gtcbsast" : 1, "gtindvpc" : 1}, 
  "charlotte"    : {"gestfips" : 37, "gtcbsa" : 16740, "gtcbsast" : 1, "gtindvpc" : 1}, 
  "seattle"      : {"gestfips" : 53, "gtcbsa" : 42660, "gtcbsast" : 1, "gtindvpc" : 1}, 
  "denver"       : {"gestfips" :  8, "gtcbsa" : 19740, "gtcbsast" : 1, "gtindvpc" : 1}, 
  "washington"   : {"gestfips" : 11, "gtcbsa" : 47900, "gtcbsast" : 1, "gtindvpc" : 1}
}

cps["city"] = ""
for city, vals in city_dict.items():
    
    query = " & ".join(["({} == {:})".format(k, v)
                        for k, v in vals.items()])
    
    cps.loc[cps.query(query).index, "city"] = city
    
cps_city = cps[cps.city != ""].copy()

In [6]:
cbsa = ["new_york", "los_angeles", "chicago", "dallas-fort_worth", "houston", 
        "philadelphia", "washington", "miami", "atlanta", "boston", 
        "san_francisco", "riverside", "phoenix", "detroit", "seattle", 
        "minneapolis", "san_diego", "tampa", "st_louis", "baltimore"]

cbsa_dict = {
    35620 : "new_york", 31080 : "los_angeles", 16980 : "chicago", 19100 : "dallas-fort_worth", 26420 : "houston", 
    37980 : "philadelphia", 47900 : "washington", 33100 : "miami", 12060 : "atlanta", 14460 : "boston", 
    41860 : "san_francisco", 40140 : "riverside", 38060 : "phoenix", 19820 : "detroit", 42660 : "seattle", 
    33460 : "minneapolis", 41740 : "san_diego", 45300 : "tampa", 41180 : "st_louis", 12580 : "baltimore"
}

cps["cbsa"] = ""
cps.loc[cps.query("gtcbsa in @cbsa_dict").index, "cbsa"] = \
   cps.query("gtcbsa in @cbsa_dict").gtcbsa.replace(cbsa_dict)

cps_cbsa = cps[cps.cbsa != ""].copy()

In [7]:
# cps[["city", "gestfips", "gtcbsa", "gtcsa", "gtmetsta", "gtcbsast", "gtindvpc", "gtco"]]\
#    .query("city != ''").drop_duplicates().sort_values("city")

These populations seem about right... Washington DC is _way_ oversampled... weights are **necessary**.  

In particular, San Francisco, where I had issues below is about 10% off of its "nominal" population (800k instead of 880k).

In [8]:
# cps_city.groupby("city").pwsswgt.sum().sort_values(ascending = False)

Internet in home.

In [9]:
def weighted_mean(G, w): return G._get_numeric_data().multiply(G[w],  axis=0).sum()/G[w].sum()

def access(df, grouper):
    
    hwss_vars =  df.query("isHouseholder").groupby(grouper).apply(weighted_mean, w = "hwhhwgt")\
                   [["H_mobdat", "H_highsp"]]
        
    hwss_vars_use =  df.query("isHouseholder & (H_at_home == False)")\
                       .groupby(grouper).apply(weighted_mean, w = "hwhhwgt")\
                       [["H_nonuse_cost", "H_nonuse_no_interest", "H_nonuse_not_available"]]
    
    pwss_vars =  df.query("isPerson").groupby(grouper).apply(weighted_mean, w = "pwsswgt")\
                   [["P_at_home", "P_at_work", "P_anywhere", "P_smartphone", "P_computer"]] 

    pwpr_vars =  df.query("isRespondent").groupby(grouper).apply(weighted_mean, w = "pwprmwgt")\
                   [["P_email", "P_text", "P_social", "P_video", "P_services"]]
        
    complete = pd.concat([pwss_vars, pwpr_vars, hwss_vars, hwss_vars_use], axis = 1).round(3)
        
    complete = complete[["P_at_home", "P_anywhere", "H_mobdat", "H_highsp", "P_at_work", 
                         "P_smartphone", "P_computer", 
                         "P_email", "P_text", "P_social", "P_video", "P_services", 
                         "H_nonuse_cost", "H_nonuse_no_interest", "H_nonuse_not_available"]] 
    
    complete.sort_values(by = "P_at_home", inplace = True)
    
    complete.index = complete.index.str.replace("_", " ").str.title()

    return complete
    

In [10]:
access(cps, "state").style.background_gradient(cmap = "viridis")

,P_at_home,P_anywhere,H_mobdat,H_highsp,P_at_work,P_smartphone,P_computer,P_email,P_text,P_social,P_video,P_services,H_nonuse_cost,H_nonuse_no_interest,H_nonuse_not_available
state,,,,,,,,,,,,,,,
North Carolina,0.682,0.743,0.655,0.631,0.279,0.64,0.523,0.863,0.89,0.718,0.718,0.248,0.225,0.577,0.012
Mississippi,0.688,0.759,0.695,0.539,0.226,0.643,0.461,0.824,0.913,0.775,0.662,0.135,0.191,0.605,0.07
West Virginia,0.692,0.773,0.651,0.637,0.24,0.609,0.528,0.847,0.865,0.741,0.708,0.133,0.223,0.597,0.033
Tennessee,0.693,0.762,0.718,0.632,0.287,0.639,0.572,0.859,0.925,0.737,0.685,0.291,0.266,0.556,0.031
Nevada,0.698,0.778,0.738,0.7,0.3,0.656,0.567,0.925,0.927,0.709,0.681,0.374,0.114,0.616,0.048
Louisiana,0.699,0.763,0.707,0.597,0.278,0.668,0.493,0.85,0.933,0.752,0.735,0.225,0.186,0.602,0.047
New Mexico,0.705,0.785,0.716,0.642,0.276,0.658,0.512,0.894,0.898,0.686,0.748,0.322,0.153,0.729,0.005
Florida,0.707,0.773,0.706,0.697,0.295,0.664,0.577,0.928,0.927,0.732,0.696,0.306,0.162,0.664,0.011
Kentucky,0.707,0.785,0.706,0.582,0.304,0.664,0.525,0.855,0.928,0.766,0.754,0.217,0.206,0.531,0.034


In [11]:
access(cps_city, "city").style.background_gradient(cmap = "viridis", axis = 0)

,P_at_home,P_anywhere,H_mobdat,H_highsp,P_at_work,P_smartphone,P_computer,P_email,P_text,P_social,P_video,P_services,H_nonuse_cost,H_nonuse_no_interest,H_nonuse_not_available
city,,,,,,,,,,,,,,,
Indianapolis,0.597,0.647,0.543,0.543,0.275,0.568,0.414,0.883,0.955,0.66,0.703,0.355,0.187,0.677,0
San Francisco,0.613,0.634,0.621,0.614,0.397,0.633,0.59,0.963,0.933,0.75,0.816,0.73,0.06,0.658,0.069
Houston,0.635,0.689,0.679,0.623,0.242,0.632,0.442,0.851,0.909,0.669,0.816,0.413,0.241,0.565,0
Chicago,0.647,0.73,0.652,0.6,0.296,0.639,0.5,0.879,0.914,0.705,0.795,0.45,0.134,0.646,0.027
San Antonio,0.666,0.714,0.708,0.667,0.304,0.636,0.537,0.92,0.939,0.775,0.837,0.423,0.175,0.576,0
Philadelphia,0.677,0.746,0.719,0.638,0.299,0.626,0.504,0.901,0.965,0.784,0.829,0.549,0.136,0.643,0
New York,0.686,0.737,0.712,0.689,0.288,0.64,0.526,0.894,0.922,0.742,0.733,0.463,0.142,0.635,0
Fort Worth,0.694,0.828,0.804,0.7,0.309,0.679,0.537,0.83,0.971,0.731,0.799,0.211,0.331,0.345,0.114
Los Angeles,0.701,0.763,0.767,0.621,0.301,0.722,0.563,0.885,0.927,0.773,0.794,0.532,0.203,0.554,0.039


The fact that SF is low still feels really weird.

In [12]:
sf = cps.query("(gestfips == 6) & (gtcbsa == 41860) & (gtcbsast == 1) & (gtco == 75) & (gtindvpc == 1)")
sf.query("heinhome == 1").hwhhwgt.sum() / sf.query("heinhome > 0").hwhhwgt.sum()

0.6602834922747424

Note that if there is no internet connection, the high speed questions don't get asked!!  Pay attention to the Universe!!

In [13]:
access(cps_cbsa, "cbsa").style.background_gradient(cmap = "viridis", axis = 0)

,P_at_home,P_anywhere,H_mobdat,H_highsp,P_at_work,P_smartphone,P_computer,P_email,P_text,P_social,P_video,P_services,H_nonuse_cost,H_nonuse_no_interest,H_nonuse_not_available
cbsa,,,,,,,,,,,,,,,
Los Angeles,0.684,0.744,0.74,0.646,0.271,0.677,0.532,0.883,0.93,0.754,0.772,0.482,0.198,0.566,0.038
Detroit,0.689,0.733,0.673,0.634,0.285,0.649,0.555,0.89,0.923,0.698,0.701,0.302,0.122,0.669,0
Miami,0.702,0.76,0.719,0.696,0.31,0.679,0.584,0.907,0.944,0.711,0.653,0.347,0.147,0.702,0
Houston,0.711,0.756,0.735,0.666,0.288,0.673,0.53,0.905,0.937,0.725,0.773,0.332,0.246,0.581,0.021
Riverside,0.715,0.765,0.753,0.702,0.264,0.683,0.52,0.877,0.927,0.775,0.743,0.324,0.211,0.638,0.023
Tampa,0.718,0.785,0.603,0.746,0.319,0.659,0.601,0.949,0.873,0.718,0.671,0.286,0.262,0.554,0.025
New York,0.729,0.779,0.717,0.731,0.312,0.679,0.577,0.904,0.926,0.719,0.713,0.409,0.151,0.627,0.007
San Diego,0.731,0.806,0.692,0.695,0.335,0.707,0.62,0.912,0.927,0.736,0.78,0.523,0.229,0.541,0.076
Boston,0.732,0.778,0.746,0.726,0.359,0.691,0.61,0.931,0.927,0.74,0.76,0.544,0.144,0.609,0.048


In [14]:
why_no_net = {
  1  : "Don't need it or not interested",
  2  : "Can't afford it",
  3  : "Not worth the cost",
  4  : "Can use it elsewhere",
  5  : "Not available in area",
  6  : "No device",
  7  : "Privacy or security concerns",
  8  : "Safety concerns",
  9  : "Moving",
  10 : "Other",
  -1 : "NA"
}

cps["why_no_net"] = cps.heprinoh.replace(why_no_net)

cps.query("heprinoh > 0").why_no_net.value_counts()

Don't need it or not interested    11188
Can't afford it                     3257
Other                               1707
Not available in area                831
Can use it elsewhere                 646
Not worth the cost                   520
No device                            478
Privacy or security concerns         299
Moving                                97
Safety concerns                       77
Name: why_no_net, dtype: int64

In [15]:
# print([x for x in cps.columns if "wgt" not in x])

### CSA doesn't work.... 

Missing: 288 houston, 420 phoenix, 476 st louis

In [16]:
csa = {"new_york" : 408, "los_angeles" : 348, "chicago" : 176, "washington" : 548, "san_jose" : 488, 
       "boston" : 148, "dallas-fort_worth" : 206, "houston" : 288, "philadelphia" : 428, "miami" : 370,
       "atlanta" : 122, "detroit" : 220, "phoenix" : 429, "seattle" : 500, "orlando" : 422, 
       "minneapolis" : 378, "denver" : 216, "cleveland" : 184, "portland" : 440, "st_louis" : 476}